In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
import cv2

image = cv2.imread('/content/20231003_140634.jpg')

org_image_wt = image.shape[1]
org_image_ht = image.shape[0]
# print(org_image_wt)
# print(org_image_ht)

AttributeError: 'NoneType' object has no attribute 'shape'

In [4]:
import json
import os
import cv2
import shutil
import numpy as np

# Set the input and output directories
input_dir = '/content/drive/MyDrive/wound_detection (1)'
output_dir = '/content/drive/MyDrive/wound_detection/wdd_temp/prepared_data'

# Create the output directory structure
os.makedirs(os.path.join(output_dir, 'train', 'images'), exist_ok=True)
os.makedirs(os.path.join(output_dir, 'train', 'labels'), exist_ok=True)
os.makedirs(os.path.join(output_dir, 'val', 'images'), exist_ok=True)
os.makedirs(os.path.join(output_dir, 'val', 'labels'), exist_ok=True)
os.makedirs(os.path.join(output_dir, 'test', 'images'), exist_ok=True)
os.makedirs(os.path.join(output_dir, 'test', 'labels'), exist_ok=True)

# Define the train, val, and test split ratios
train_ratio = 0.7
val_ratio = 0.2
test_ratio = 0.1

# Function to resize image to 224x224
def resize_image(image):
    return cv2.resize(image, (224, 224))

# Iterate over the input subdirectories
for main_subdir in os.listdir(input_dir):
    if os.path.isdir(os.path.join(input_dir, main_subdir)):
        for subdir in os.listdir(os.path.join(input_dir, main_subdir)):
            if os.path.isdir(os.path.join(input_dir, main_subdir, subdir)):
                for filename in os.listdir(os.path.join(input_dir, main_subdir, subdir)):
                    if filename.endswith('.json'):
                        # Load the JSON annotation file
                        with open(os.path.join(input_dir, main_subdir, subdir, filename), 'r') as f:
                            data = json.load(f)

                        # Extract the image filename and ROI coordinates
                        image_filename = data['imagePath']
                        image_path = os.path.join(input_dir, main_subdir, subdir, image_filename.replace('.json', '.jpg'))
                        roi_coords = data['shapes'][0]['points']

                        # Check if the image file exists
                        if os.path.exists(image_path):
                            try:
                                # Load the image
                                image = cv2.imread(image_path)
                                image = resize_image(image)

                                # Determine the split set (train, val, or test)
                                split_set = 'train'
                                rand_value = np.random.random()
                                if rand_value < val_ratio:
                                    split_set = 'val'
                                elif rand_value < val_ratio + test_ratio:
                                    split_set = 'test'

                                # Save the resized image
                                cv2.imwrite(os.path.join(output_dir, split_set, 'images', image_filename.replace('.json', '.jpg')), image)

                                # Generate the YOLO format label
                                label_filename = os.path.splitext(image_filename)[0] + '.txt'
                                label_path = os.path.join(output_dir, split_set, 'labels', label_filename)
                                with open(label_path, 'w') as f:
                                    # Assuming a single object (roi) in the image
                                    x_center = (roi_coords[0][0] + roi_coords[1][0]) / 2
                                    y_center = (roi_coords[0][1] + roi_coords[1][1]) / 2
                                    width = abs(roi_coords[1][0] - roi_coords[0][0])
                                    height = abs(roi_coords[1][1] - roi_coords[0][1])
                                    # Normalize coordinates
                                    x_center /= org_image_wt
                                    y_center /= org_image_ht
                                    width /= org_image_wt
                                    height /= org_image_ht
                                    f.write(f'0 {x_center} {y_center} {width} {height}')

                            except cv2.error as e:
                                print(f"Error processing {image_path}: {e}")
                            except Exception as e:
                                print(f"Unexpected error processing {image_path}: {e}")
                        else:
                            print(f"Image file not found: {image_path}")

print('Dataset preparation complete!')


Dataset preparation complete!


In [ ]:
!pip install ultralytics==8.0.20

from IPython import display
display.clear_output()

import ultralytics
ultralytics.checks()

In [ ]:
from ultralytics import YOLO
# %cd /content/drive/MyDrive/wound_detection/wdd_temp
# !yolo task=detect mode=train model=yolov8s.pt data= data.yaml epochs=25 imgsz=224 plots=True

In [ ]:
# !yolo task=detect mode=predict model=/content/drive/MyDrive/wound_detection/wdd_temp/runs/detect/train/weights/best.pt conf=0.25 source=/content/drive/MyDrive/wound_detection/wdd_temp/prepared_data/test/images save=True

In [ ]:
# !yolo task=detect mode=predict model=/content/drive/MyDrive/wound_detection/wdd_temp/runs/detect/train/weights/best.pt conf=0.05 source=/content/drive/MyDrive/example save=True

In [ ]:
import glob
from PIL import Image
from IPython.display import Image,display

for image_path in glob.glob('/content/runs/detect/predict5/*.jpg'):
  display(Image(filename=image_path, width=600))
  print('\n')

In [ ]:
from IPython.display import Image, display

# Display the image
display(Image(filename='/content/runs/detect/predict4/20221102_224054.jpg'))

In [ ]:
from ultralytics import YOLO
model = YOLO("yolov8n-seg.pt")
model.predict(source='/content/drive/MyDrive/wound_detection/wdd_temp/admsc/admsc_day0/20231003_140634.jpg')